# REDUCEDHEATCARB analysis

This JupyterLabs notebook can be used for physics ifnormed machine learning analysis in the REDUCEDHEATCAB project.
Don't forget to install the requirements listed in [requirements.txt](../requirements.txt) first!

## Setting the stage

First several imports and variables need to be defined


### Imports and generic settings

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib widget


from tqdm.notebook import tqdm

from gekko import GEKKO

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')
from nfh_utils import *

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

%load_ext autoreload

from preprocessor import Preprocessor
from rhc_analysis import Learner

%matplotlib inline
%matplotlib widget

from plotter import Plot



### Reading preprocessed interpolated properties from a parquet file

In [ ]:
%%time

rhc_preprocessed_poperties_file='rhc_preprocessed_properties.parquet'

# Attempt to read the Parquet file
try:
    df_prep = pd.read_parquet(
        rhc_preprocessed_poperties_file, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

In [ ]:
#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFrames are properly sorted
if not df_prep.index.is_monotonic_increasing:
    print('df needed index sorting')
    df_prep = df_prep.sort_index()  

In [ ]:
print(f"df_prep.count().sum(): {df_prep.count().sum():_}")

In [ ]:
df_prep.info()

In [ ]:
print("[\n'","', \n'".join(sorted(df_prep.columns)),"'\n]")

In [ ]:
%autoreload 2
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'ppm' : r'$ppm$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'p' : r'$persons$',
    'W' : r'$W$',
    'W_m_2' : r'$W/m^{2}$',
    'm_s_1' : r'$m/s$'    
}

In [ ]:
# visuaize all input data
df_plot = df_prep

In [ ]:
#Plot all properties from all sources for all ids
# Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)

### Calculate additional properties 

In [ ]:
%%time
df_prep['calculated_e_use__W'] = df_prep['device_p1-reader_e_use_hi__W'] + df_prep['device_p1-reader_e_use_lo__W']
df_prep['calculated_e_ret__W'] = df_prep['device_p1-reader_e_ret_hi__W'] + df_prep['device_p1-reader_e_ret_lo__W'] 


In [ ]:
%%time
# Calculate Q_gain_ch__W based on national average efficiency
df_prep['calculated_Q_gain_ch__W'] = df_prep['batch_import_remeha_g_use_ch_lhv__W'] *  g_groningen_hhv___MJ_m_3 /  g_groningen_lhv___MJ_m_3 * eta_ch_nl_avg_hhv__W0

## Learn parameters

Most of the heavy lifting is done by the `learn_home_parameters()` function, which again uses the [GEKKO Python](https://machinelearning.byu.edu/) dynamic optimization toolkit.

In [ ]:
# set room metadata to None, then learn_home_parameters() will derive the metadata from the ids.

hints = {
    'A_sol__m2': 12.0,                                      # initial estimate for apparent solar aperture
    'eta_ch_hhv__W0' : eta_ch_nl_avg_hhv__W0,           # average home heating efficiency of a gas boiler (superior value)
    'eta_not_ch_hhv__W0' : 0.34,                        # average home heating efficiency indirecly DHW & cooking (superior value)
    'g_not_ch_hhv__W' : g_not_ch_nl_avg_hhv__W,         # average gas power (heating value) for other purposes than home heating [W] 
    'occupancy__p' : (2.2 * 7.7/24),                    # average house occupancy (2.2 persons, 7.7 of 24 hours)
    'Q_gain_int__W_p_1' : (77 * 8.6/24 + 105 * 7.7/24), # average heat gain per occupant (77 W for 8.6 hours, 105 W for 7.7 hours)
    'wind_chill__degC_s_m_1' : 0.67                     # average for Dutch homes, according to KNMI: https://cdn.knmi.nl/knmi/pdf/bibliotheek/knmipubmetnummer/knmipub219.pdf 
}

learn = ['A_sol__m2']

#select column names
property_sources = {
    'temp_in__degC' : 'batch_import_remeha_temp_in__degC',
    'temp_out__degC' : 'batch_import_KNMI_temp_out__degC',
    'wind__m_s_1' : 'batch_import_KNMI_wind__m_s_1',
    'ghi__W_m_2' : 'batch_import_KNMI_ghi__W_m_2', 
    'g_use_hhv__W' : 'device_p1-reader_g_use_hhv__W',
    'e_use__W' : 'calculated_e_use__W',
    'e_ret__W' : 'calculated_e_ret__W'
}

In [ ]:
# learn the model parameters and write results to a dataframe
%autoreload 2
df_results_per_period, df_results = Learner.learn_home_parameters(df_prep, 
                                                                  property_sources = property_sources, 
                                                                  learn = learn, 
                                                                  hints = hints,
                                                                  ev_type = 2
                                                                 )

### Result per learning period

In [ ]:
df_results_per_period

In [ ]:
# show essential statistics for the learned values
df_stats = df_results_per_period.describe().filter(regex='^actual_|^learned_')
df_stats.loc[df_stats.index.get_level_values(0).isin(['mean', 'std', 'min', 'max'])]

In [ ]:
# show essential statistics for the errors
df_stats = df_results_per_period.describe().filter(regex='^mae_|^rmse')
df_stats.loc[df_stats.index.get_level_values(0).isin(['mean', 'std', 'min', 'max'])]

In [ ]:
# show essential statistics for the error values, per id
df_stats = df_results_per_period.groupby('id').describe().stack().filter(regex='^mae_|^rmse')
df_stats.loc[df_stats.index.get_level_values(1).isin(['mean', 'std', 'min', 'max'])]

### Result Visualization

In [ ]:
df_results

In [ ]:
df_plot = df_prep[[prop for prop in df_prep.columns.values if prop.split('__')[-1] == 'degC']]

In [ ]:
#Plot only temperatures from all sources for all ids
Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)

In [ ]:
df_lot = df_prep

In [ ]:
#Plot all properties from all sources for all ids
Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)